# How to subset cloud-hosted Hubble FITS files using AstroPy?

*Prepared by Geert Barentsen, last updated on March 17, 2022.*

As part of the Fornax project, we aim to identify what changes to the FITS format and/or FITS libraries are required to enable astronomers to analyze cloud-hosted FITS files in a seamless and efficient way.  To help inform our decision making, this notebook explores how a Hubble Space Telescope image hosted in AWS S3 can best be analyzed using Python tools that exist today.

## Goals

This notebook aims to address three questions:
* How can we load S3-hosted FITS files using existing Python tools, specifically AstroPy?
* What is the current behavior of these tools?
* How can we improve the performance?

The notebook will start by focusing on the problem of extracting a single Header Data Unit (HDU) from a large cloud-hosted FITS file.  This will be followed by a discussion on slicing data from a single HDU.

## 1. Benchmark data set

For demonstration purposes, we selected a single 214 MB FITS file containing an HST/ACS image obtained as part of the *COSMOS 2-Degree ACS Survey* near coordinates (ra, dec) = (150.421375, 2.422383).  I selected Hubble data because it is easy to interpret and readily available on S3.  Everything in this notebook should apply to FITS files from any project however.

For simplicity, we will assume that the user has already discovered the data set (e.g. using astroquery) and has obtained the following file URIs:

In [ ]:
# HTTP URL of the FITS file at MAST
http_uri = "https://mast.stsci.edu/api/v0.1/Download/file/?uri=mast:HST/product/j8pu0y010_drc.fits"

# AWS S3 URI of the same FITS file
s3_bucket = "stpubdata"
s3_key = "hst/public/j8pu/j8pu0y010/j8pu0y010_drc.fits"
s3_uri = f"s3://{s3_bucket}/{s3_key}"

# Location of the data on local disk
local_path = "/tmp/j8pu0y010_drc.fits"

We will use the following Python imports below:

In [ ]:
import os

from astropy.io import fits
from matplotlib import pyplot as plt

# boto3 is the official AWS SDK for Python
import boto3
from botocore import UNSIGNED
from botocore.config import Config

## 2. Baseline benchmark: downloading the entire file prior to opening

At present, users can already pass HTTP urls to `astropy.io.fits.open(url)`.  AstroPy will recognize the url and download the entire file to the local filesystem before opening the file.

AstroPy can conceivably be modified to adopt the same behavior for S3 URIs.  We will benchmark that behavior here by downloading the file using the AWS `boto3` library and subsequently opening the file using AstroPy.

In [ ]:
%%time
# How long does it take to download the file?
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))
s3.download_file(s3_bucket, s3_key, local_path)

In [ ]:
# How much data did we download?
size = os.path.getsize(local_path) / (1024*1024)
print(f"Downloaded {size:.0f} MB")

Having downloaded the FITS file, it is trivial to open the file using AstroPy and plot a subset contained in HDU #1:

In [ ]:
# Plot a subset of the image contained in HDU 1
def plot_subset(hdulist):
    plt.imshow(hdulist[1].data[1600:1950, 2000:2250], vmin=0., vmax=0.1);

hdulist = fits.open(local_path)
plot_subset(hdulist)

## 3. Can we avoid downloading the entire file?

The baseline benchmark is inefficient because the entire file was downloaded, which is unnecessary because we only plotted data from HDU #1.  Using `hdulist.info()` we can observe that the FITS file contains three additional large HDU's which we did not require:

In [ ]:
hdulist.info()

How much unnecessary data did we download?  Let's have a look at the byte structure of the FITS file:

In [ ]:
from fornaxutils import print_byte_layout  # custom function defined in fornax.py
print_byte_layout(hdulist)

The table above shows that HDU #1 is 71 MB in size.  We can improve on the baseline benchmark by a factor ~3 by downloading this 71 MB section rather than the entire 214 MB file.  The table also shows the byte positions of the header (`hdrLoc`) and data section (`dataLoc`) of each HDU for future reference.

## 4. How to download a single HDU using AstroPy?

For local files, AstroPy supports reading only the required HDUs via the `lazy_load_hdus` configuration parameter provided by [astropy.io.fits.open](https://docs.astropy.org/en/stable/io/fits/api/files.html#astropy.io.fits.open).  This mode is enabled by default and prevents AstroPy from loading HDU data into memory until it is required by the user.

Specifically:
* The header of a HDU is only read into memory once `HDUList.__getitem__()` is called for that HDU.
* The data of a HDU is only read once the `.data` property of an HDU object is accessed (i.e., `.data` is implemented using the `@lazyproperty` decorator).


AstroPy implements this feature using the ability of the Python file object to read sub-sections of data using standard OS random file access operations.  For example, we can read a single header keyword from a local file as follows:

In [ ]:
f = open(local_path)
f.seek(560) # move to byte position 560
f.read(80)  # read 80 bytes

Historically such random access file operations could only be used on local files.  In recent years, identical file object APIs have been created to enable seamless random access to remote file systems (including AWS S3, Google Cloud Storage, Azure Blob Storage, etc). These APIs have been grouped in the [fsspec project](https://filesystem-spec.readthedocs.io/en/latest/) which has its roots within the [Dask project](https://dask.org/).

For example, we can efficiently read the same bytes from the FITS file using [s3fs](https://s3fs.readthedocs.io/en/latest/), which is the fsspec-based package to access data from AWS s3:

In [ ]:
from s3fs import S3FileSystem

s3fs = S3FileSystem(anon=True)
with s3fs.open(s3_uri, mode="rb") as fh:
    fh.seek(560)
    print(fh.read(79))

Behind the scenes, `s3fs` uses the AWS SDK to fetch data using byte-range GET requests.  To observe the exact behavior I created a custom `NetworkProfiler` tool which monkey-patches fsspec to reveal the data transfer behavior (shown as red text):

In [ ]:
from fornaxutils import NetworkProfiler   # custom class defined in fornaxutils.py

with NetworkProfiler() as profiler:
    with s3fs.open(s3_uri, mode="rb", block_size=80, cache_type="block") as fh:
        fh.seek(560)
        fh.read(79)
    profiler.summary()

**Important**: the `block_size` and `cache_type` parameters passed to fsspec's `open()` function play a critical role in determining the performance.  These parameters control the size and frequency of the underlying GET requests that will be made to the remote server, similar to the way in which an operating system's *page size* impacts the efficency of swapping data between disk and memory.  The ideal values of these settings will depend on the use case, the network latency, and the server throughput.

Note that we can access files exposed by the MAST HTTP server in the same way by substituting `S3FileSystem` with `HTTPFileSystem`. For example:

In [ ]:
from fsspec.implementations.http import HTTPFileSystem

with NetworkProfiler() as profiler:
    httpfs = HTTPFileSystem()
    with httpfs.open(http_uri, mode='rb', block_size=80, cache_type="block") as fh:
        fh.seek(560)
        fh.read(79)
    profiler.summary()

Other available options offered by fsspec include `GCSFileSystem` (Google Cloud), `AzureBlobFileSystem`, `HadoopFileSystem`, `FTPFileSystem`, `LocalFileSystem`, and many more.

## 5. Can AstroPy access a single HDU from S3 storage in this way?

Yes!  By passing the `S3File` object provided by `s3fs`, we can trick AstroPy into thinking that an S3-hosted file is stored locally and supports random access. For example, we can read the header of HDU #4 by transferring only a tiny fraction of the FITS file:

In [ ]:
with NetworkProfiler() as profiler:
    s3fs = S3FileSystem(anon=True)
    with s3fs.open(s3_uri, mode="rb", block_size=100_000, cache_type="block") as fh:
        with fits.open(fh) as hdulist:
            hdr = hdulist[4].header
    profiler.summary()

Next, let's try accessing data by plotting the same cutout from HDU 1 as we did before:

In [ ]:
with NetworkProfiler() as profiler:
    s3fs = S3FileSystem(anon=True)
    with s3fs.open(s3_uri, mode="rb", block_size=25_000_000, cache_type="block") as fh:
        with fits.open(fh) as hdulist:
            plot_subset(hdulist)
    profiler.summary()

As expected, AstroPy is able to access the image stored in HDU #1 without requiring the other image extensions to be loaded (i.e., only ~71 MB was downloaded rather than 214 MB).  Unfortunately, AstroPy does load the entire data section of HDU #1 rather than downloading only the cutout.

# 6. Can we slice the remote image data using AstroPy?

We would like to limit the data transfer to the bytes needed to plot the cutout region, rather than downloading the entire image array for HDU #1 (71 MB), 

AstroPy supports reading slices of data via the [memory mapping mechanism](https://docs.astropy.org/en/stable/io/fits/index.html#working-with-large-files) and the [buffer protocol](https://docs.python.org/3/c-api/buffer.html). Specifically, as part of `astropy.io.fits.file._File.readarray()`, AstroPy initializes data arrays by passing a `mmap` object to the `buffer` parameter of `np.ndarray` as follows: 
```python
return np.ndarray(shape=shape, dtype=dtype, offset=offset, buffer=self._mmap)
```

By creating an array in this way, numpy slicing will be translated into file seek/read calls at runtime as needed.  Unfortunately this happens deep inside the Numpy C code (or perhaps even the OS kernel), which makes it difficult to plug the pure-Python file interfaces provided by fsspec into this mechanism.  The buffer protocol is a Python C API concept which cannot be implemented in pure Python. As a result, it seems unlikely that we can repurpose the buffer interface mechanism here.

Rather than returning `np.ndarray(buffer=....)` we will have to figure out how AstroPy may be modified to return a custom array-like object which can translate slicing operations into s3fs-based seek/read calls as needed. It is likely that we may be able to borrow code or insights from the array objects provided by Dask or Xarray, but this needs further research.

## Conclusions

1. By passing file-like objects created using the `fsspec/s3fs` library into `astropy.io.fits.open()`, we can already use standard AstroPy features to extract individual HDU extensions from cloud-hosted FITS files in an efficient way.  The header and data sections of HDUs are only transferred from S3 when needed thanks to the existing `lazy_load_hdus` feature of AstroPy. We can start documenting and advertising the use of `s3fs` with AstroPy in this way -- this strategy is already more efficient than downloading entire files for applications which require a single HDU from a large file.

2. Extracting slices of data from a single HDU is not yet possible and will likely require modifications to AstroPy.  Specifically, we need to modify AstroPy's `_File.readarray()` method to return an array-like object that can translate slicing operations into S3 file seek/read calls on the fly.

3. We may not gain much by creating a "central directory" of header information and byte offsets, because AstroPy is already smart about loading HDU header and data sections only when they are needed.


## Footnotes

* The `fitsio` package is a popular alternative to AstroPy.  It is unlikely to work with `fsspec`-based libraries because it is implemented by wrapping the `cfitsio` library. In contrast, `astropy.io.fits` is pure Python with the exception of the compression algorithms.
* HDF5 v1.10.6 and later supports cloud-hosted files in a similar way by providing the [S3 Virtual File Driver](https://portal.hdfgroup.org/display/HDF5/H5P_SET_FAPL_ROS3) (a role fulfilled by `s3fs` in our case).  Like AstroPy, it does not appear to support slicing arrays and it does not benefit from a central directory of metadata.
* The `zarr` Python library natively support slicing arrays hosted in cloud-hosted zarr files.